In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

spx = pd.read_csv(
    "data/SPX.csv",
    sep=";",
    usecols=["Date", "Close"],
    parse_dates=["Date"]
)
spx["Date"] = pd.to_datetime(spx["Date"], format="mixed", dayfirst=False, errors="coerce")




spx = spx[["Date", "Close"]].rename(columns={"Close": "Price"})
spx = spx.sort_values("Date")


# Nur die ersten beiden Spalten einlesen, Header manuell setzen
kgv = pd.read_csv(
    "KGV.csv",
    sep=";",             # fest auf Semikolon setzen
    skiprows=1,          # erste Zeile ("Tabelle 1") überspringen
    usecols=[0, 1],      # nur Datum und KGV
    names=["Date", "PE Ratio"],  # eigene Spaltennamen
    header=None
)

# Datentypen bereinigen
kgv["Date"] = pd.to_datetime(kgv["Date"], errors="coerce")
kgv["PE Ratio"] = pd.to_numeric(kgv["PE Ratio"], errors="coerce")

# Aufräumen
kgv = kgv.dropna().sort_values("Date")


# Sicherstellen, dass Datumsspalten korrekt sind
spx["Date"] = pd.to_datetime(spx["Date"], format="mixed", dayfirst=False, errors="coerce")
spx["Price"] = spx["Price"].astype(str).str.replace(",", "").astype(float)

kgv["Date"] = pd.to_datetime(kgv["Date"])

# 1. Vollständigen täglichen Zeitraum erzeugen (von min bis max in SPX)
full_dates = pd.DataFrame({"Date": pd.date_range(start=spx["Date"].min(), end=spx["Date"].max(), freq="D")})

# 2. KGV auf vollen Datumsbereich mergen
kgv_full = full_dates.merge(kgv, on="Date", how="left")

# 3. Interpolation und Extrapolation (linear in beide Richtungen)
kgv_full["PE Ratio"] = kgv_full["PE Ratio"].interpolate(method="linear", limit_direction="both")

# 4. Auf SPX-Daten reduzieren (nur Tage mit Kursdaten)
kgv_on_spx_days = spx.merge(kgv_full, on="Date", how="left")

# 5. Ergebnis anzeigen
print(kgv_on_spx_days.tail())

In [ ]:
import pandas as pd

# Annahme: df ist dein bereits vorbereiteter DataFrame
df = kgv_on_spx_days.copy()
df = df.iloc[14600:].reset_index(drop=True)
# 1. 200-Tage-Linie berechnen
df["SMA_200"] = df["Price"].rolling(window=200).mean()

# 2. Tägliche Rendite berechnen
df["Daily Return"] = df["Price"].pct_change()

# 3. Portfoliowert berechnen
initial_value = 100
portfolio = [initial_value]  # erster Wert ist immer 100

for i in range(1, len(df)):
    if i < 200:
        # Noch nicht genug Daten für SMA – Portfolio bleibt gleich
        portfolio.append(initial_value)
    else:
        price_yesterday = df.iloc[i - 1]["Price"]
        sma_yesterday = df.iloc[i - 1]["SMA_200"]
        ret_today = df.iloc[i]["Daily Return"]

        if price_yesterday > sma_yesterday:
            # Investiert mit 3x Hebel
            new_value = portfolio[-1] * (1 + 3 * ret_today)
        else:
            # Konstantes Wachstum (z. B. Geldmarktzins o. Ä.)
            new_value = portfolio[-1] * (1 + 0.0001)

        portfolio.append(new_value)

# 4. Neue Spalte anhängen
df["Portfolio Value"] = portfolio

# 5. Rückgabe
df.tail(10)
#return:100*1,171^96,5

### S&P vs Nasdaq 100


In [107]:
import pandas as pd

# CSV einlesen (ohne converters)
df = pd.read_csv("data/SPX&Nasdaq.csv", sep=";", parse_dates=["Date"])

# Falls SPX/Price nicht float sind, dann hier korrekt umwandeln:
# z.B. von '3,485.70' → 3485.70
if df["SPX"].dtype == object:
    df["SPX"] = df["SPX"].str.replace(",", "").astype(float)

if df["Price"].dtype == object:
    df["Price"] = df["Price"].str.replace(",", "").astype(float)

# --- SPX STRATEGIE ---
spx_df = df[["Date", "SPX"]].copy()

spx_df["SMA_200"] = spx_df["SPX"].rolling(window=200).mean()
spx_df["Daily Return"] = spx_df["SPX"].pct_change()

strategy_return_spx = []
for i in range(len(spx_df)):
    if i == 0 or pd.isna(spx_df.iloc[i - 1]["SMA_200"]):
        strategy_return_spx.append(0)
    else:
        price_yesterday = spx_df.iloc[i - 1]["SPX"]
        sma_yesterday = spx_df.iloc[i - 1]["SMA_200"]
        daily_ret = spx_df.iloc[i]["Daily Return"]

        if price_yesterday > sma_yesterday:
            strategy_return_spx.append(3 * daily_ret)
        else:
            strategy_return_spx.append(0)

spx_df["Strategy Return"] = strategy_return_spx
spx_df["Portfolio Value SPX"] = (1 + spx_df["Strategy Return"]).cumprod() * 100

# --- NASDAQ STRATEGIE ---
nasdaq_df = df[["Date", "Price"]].copy()

nasdaq_df["SMA_200"] = nasdaq_df["Price"].rolling(window=100).mean()
nasdaq_df["Daily Return"] = nasdaq_df["Price"].pct_change()

strategy_return_nasdaq = []
for i in range(len(nasdaq_df)):
    if i == 0 or pd.isna(nasdaq_df.iloc[i - 1]["SMA_200"]):
        strategy_return_nasdaq.append(0)
    else:
        price_yesterday = nasdaq_df.iloc[i - 1]["Price"]
        sma_yesterday = nasdaq_df.iloc[i - 1]["SMA_200"]
        daily_ret = nasdaq_df.iloc[i]["Daily Return"]

        if price_yesterday > sma_yesterday:
            strategy_return_nasdaq.append(3 * daily_ret)
        else:
            strategy_return_nasdaq.append(0)

nasdaq_df["Strategy Return"] = strategy_return_nasdaq
nasdaq_df["Portfolio Value Nasdaq"] = (1 + nasdaq_df["Strategy Return"]).cumprod() * 100

# --- Ergebnis anzeigen ---
print("SPX:\n", spx_df[["Date", "SPX", "Portfolio Value SPX"]].tail(10))
print("\nNasdaq:\n", nasdaq_df[["Date", "Price", "Portfolio Value Nasdaq"]].tail(10))




SPX:
             Date      SPX  Portfolio Value SPX
9821  03/24/2025  5718.08         42077.116846
9822  03/25/2025  5760.42         42077.116846
9823  03/26/2025  5694.41         40630.602278
9824  03/27/2025  5670.94         40630.602278
9825  03/28/2025  5572.42         40630.602278
9826  03/31/2025  5488.73         40630.602278
9827  04/01/2025  5558.52         40630.602278
9828  04/02/2025  5571.48         40630.602278
9829  04/03/2025  5390.83         40630.602278
9830  04/04/2025  5069.90         40630.602278

Nasdaq:
             Date     Price  Portfolio Value Nasdaq
9821  03/24/2025  20180.45            467058.02905
9822  03/25/2025  20287.83            467058.02905
9823  03/26/2025  19916.99            467058.02905
9824  03/27/2025  19798.62            467058.02905
9825  03/28/2025  19281.40            467058.02905
9826  03/31/2025  19278.45            467058.02905
9827  04/01/2025  19436.42            467058.02905
9828  04/02/2025  19581.78            467058.02905
9829  04

### EMA

In [106]:
import pandas as pd

# CSV einlesen
df = pd.read_csv("data/SPX&Nasdaq.csv", sep=";", parse_dates=["Date"])

# Sicherstellen, dass SPX und Price floats sind
if df["SPX"].dtype == object:
    df["SPX"] = df["SPX"].str.replace(",", "").astype(float)

if df["Price"].dtype == object:
    df["Price"] = df["Price"].str.replace(",", "").astype(float)

# --- SPX STRATEGIE mit EMA 200 ---
spx_df = df[["Date", "SPX"]].copy()

spx_df["EMA_200"] = spx_df["SPX"].ewm(span=200, adjust=False).mean()
spx_df["Daily Return"] = spx_df["SPX"].pct_change()

strategy_return_spx = []
for i in range(len(spx_df)):
    if i == 0 or pd.isna(spx_df.iloc[i - 1]["EMA_200"]):
        strategy_return_spx.append(0)
    else:
        price_yesterday = spx_df.iloc[i - 1]["SPX"]
        ema_yesterday = spx_df.iloc[i - 1]["EMA_200"]
        daily_ret = spx_df.iloc[i]["Daily Return"]

        # Modifizierte Schwelle (97 % des EMA)
        if price_yesterday > 1 * ema_yesterday:
            strategy_return_spx.append(3 * daily_ret)
        else:
            strategy_return_spx.append(0)

spx_df["Strategy Return"] = strategy_return_spx
spx_df["Portfolio Value SPX"] = (1 + spx_df["Strategy Return"]).cumprod() * 100

# --- NASDAQ STRATEGIE mit EMA 200 ---
nasdaq_df = df[["Date", "Price"]].copy()

nasdaq_df["EMA_200"] = nasdaq_df["Price"].ewm(span=200, adjust=False).mean()
nasdaq_df["Daily Return"] = nasdaq_df["Price"].pct_change()

strategy_return_nasdaq = []
for i in range(len(nasdaq_df)):
    if i == 0 or pd.isna(nasdaq_df.iloc[i - 1]["EMA_200"]):
        strategy_return_nasdaq.append(0)
    else:
        price_yesterday = nasdaq_df.iloc[i - 1]["Price"]
        ema_yesterday = nasdaq_df.iloc[i - 1]["EMA_200"]
        daily_ret = nasdaq_df.iloc[i]["Daily Return"]

        if price_yesterday > 0.98 * ema_yesterday:
            strategy_return_nasdaq.append(3 * daily_ret)
        else:
            strategy_return_nasdaq.append(0)

nasdaq_df["Strategy Return"] = strategy_return_nasdaq
nasdaq_df["Portfolio Value Nasdaq"] = (1 + nasdaq_df["Strategy Return"]).cumprod() * 100

# --- Ergebnis anzeigen ---
print("SPX:\n", spx_df[["Date", "SPX", "Portfolio Value SPX"]].tail(10))
print("\nNasdaq:\n", nasdaq_df[["Date", "Price", "Portfolio Value Nasdaq"]].tail(10))


SPX:
             Date      SPX  Portfolio Value SPX
9821  03/24/2025  5718.08         38090.499482
9822  03/25/2025  5760.42         38936.632318
9823  03/26/2025  5694.41         37598.080390
9824  03/27/2025  5670.94         37133.189235
9825  03/28/2025  5572.42         37133.189235
9826  03/31/2025  5488.73         37133.189235
9827  04/01/2025  5558.52         37133.189235
9828  04/02/2025  5571.48         37133.189235
9829  04/03/2025  5390.83         37133.189235
9830  04/04/2025  5069.90         37133.189235

Nasdaq:
             Date     Price  Portfolio Value Nasdaq
9821  03/24/2025  20180.45            2.656774e+06
9822  03/25/2025  20287.83            2.699184e+06
9823  03/26/2025  19916.99            2.551169e+06
9824  03/27/2025  19798.62            2.505683e+06
9825  03/28/2025  19281.40            2.309307e+06
9826  03/31/2025  19278.45            2.309307e+06
9827  04/01/2025  19436.42            2.309307e+06
9828  04/02/2025  19581.78            2.309307e+06
9829  04

### SPX vs Nasdaq 0.98*200SMA

In [105]:
import pandas as pd

# CSV einlesen
df = pd.read_csv("data/SPX&Nasdaq.csv", sep=";", parse_dates=["Date"])

# Sicherstellen, dass SPX und Price floats sind
if df["SPX"].dtype == object:
    df["SPX"] = df["SPX"].str.replace(",", "").astype(float)

if df["Price"].dtype == object:
    df["Price"] = df["Price"].str.replace(",", "").astype(float)

# --- SPX STRATEGIE ---
spx_df = df[["Date", "SPX"]].copy()

spx_df["SMA_200"] = spx_df["SPX"].rolling(window=200).mean()
spx_df["Daily Return"] = spx_df["SPX"].pct_change()

strategy_return_spx = []
for i in range(len(spx_df)):
    if i == 0 or pd.isna(spx_df.iloc[i - 1]["SMA_200"]):
        strategy_return_spx.append(0)
    else:
        price_yesterday = spx_df.iloc[i - 1]["SPX"]
        sma_yesterday = spx_df.iloc[i - 1]["SMA_200"]
        daily_ret = spx_df.iloc[i]["Daily Return"]

        # Modifizierte Schwelle (98 % der SMA)
        if price_yesterday > 0.97 * sma_yesterday:
            strategy_return_spx.append(3 * daily_ret)
        else:
            strategy_return_spx.append(0)

spx_df["Strategy Return"] = strategy_return_spx
spx_df["Portfolio Value SPX"] = (1 + spx_df["Strategy Return"]).cumprod() * 100

# --- NASDAQ STRATEGIE ---
nasdaq_df = df[["Date", "Price"]].copy()

nasdaq_df["SMA_200"] = nasdaq_df["Price"].rolling(window=200).mean()
nasdaq_df["Daily Return"] = nasdaq_df["Price"].pct_change()

strategy_return_nasdaq = []
for i in range(len(nasdaq_df)):
    if i == 0 or pd.isna(nasdaq_df.iloc[i - 1]["SMA_200"]):
        strategy_return_nasdaq.append(0)
    else:
        price_yesterday = nasdaq_df.iloc[i - 1]["Price"]
        sma_yesterday = nasdaq_df.iloc[i - 1]["SMA_200"]
        daily_ret = nasdaq_df.iloc[i]["Daily Return"]

        if price_yesterday > 0.98 * sma_yesterday:
            strategy_return_nasdaq.append(3 * daily_ret)
        else:
            strategy_return_nasdaq.append(0)

nasdaq_df["Strategy Return"] = strategy_return_nasdaq
nasdaq_df["Portfolio Value Nasdaq"] = (1 + nasdaq_df["Strategy Return"]).cumprod() * 100

# --- Ergebnis anzeigen ---
print("SPX:\n", spx_df[["Date", "SPX", "Portfolio Value SPX"]].tail(10))
print("\nNasdaq:\n", nasdaq_df[["Date", "Price", "Portfolio Value Nasdaq"]].tail(10))
#78600

SPX:
             Date      SPX  Portfolio Value SPX
9821  03/24/2025  5718.08         69624.656544
9822  03/25/2025  5760.42         71171.281263
9823  03/26/2025  5694.41         68724.576191
9824  03/27/2025  5670.94         67874.813457
9825  03/28/2025  5572.42         64337.290601
9826  03/31/2025  5488.73         61438.520668
9827  04/01/2025  5558.52         61438.520668
9828  04/02/2025  5571.48         61868.262701
9829  04/03/2025  5390.83         55850.201975
9830  04/04/2025  5069.90         55850.201975

Nasdaq:
             Date     Price  Portfolio Value Nasdaq
9821  03/24/2025  20180.45            1.513909e+06
9822  03/25/2025  20287.83            1.538076e+06
9823  03/26/2025  19916.99            1.453733e+06
9824  03/27/2025  19798.62            1.427813e+06
9825  03/28/2025  19281.40            1.427813e+06
9826  03/31/2025  19278.45            1.427813e+06
9827  04/01/2025  19436.42            1.427813e+06
9828  04/02/2025  19581.78            1.427813e+06
9829  04

### Cross Signaling


In [99]:
import pandas as pd

# CSV einlesen
df = pd.read_csv("data/SPX&Nasdaq.csv", sep=";", parse_dates=["Date"])

# Sicherstellen, dass SPX und Price numerisch sind
if df["SPX"].dtype == object:
    df["SPX"] = df["SPX"].str.replace(",", "").astype(float)

if df["Price"].dtype == object:
    df["Price"] = df["Price"].str.replace(",", "").astype(float)

df['Price'] =df['Price'] /100
# Berechne SMAs und Returns für beide Indizes
df["SPX_SMA_200"] = df["SPX"].rolling(window=200).mean()
df["Nasdaq_SMA_200"] = df["Price"].rolling(window=200).mean()

df["SPX_Return"] = df["SPX"].pct_change()
df["Nasdaq_Return"] = df["Price"].pct_change()

# --- Strategie: Nasdaq wird gehalten, wenn SPX > SPX_SMA_200 ---
nasdaq_strategy_returns = []
for i in range(len(df)):
    if i == 0 or pd.isna(df.iloc[i - 1]["SPX_SMA_200"]):
        nasdaq_strategy_returns.append(0)
    else:
        spx_yesterday = df.iloc[i - 1]["SPX"]
        spx_sma_yesterday = df.iloc[i - 1]["SPX_SMA_200"]
        daily_ret_nasdaq = df.iloc[i]["Nasdaq_Return"]

        if spx_yesterday > spx_sma_yesterday:
            nasdaq_strategy_returns.append(3 * daily_ret_nasdaq)
        else:
            nasdaq_strategy_returns.append(0)

df["Nasdaq Strategy Return"] = nasdaq_strategy_returns
df["Portfolio Value Nasdaq"] = (1 + df["Nasdaq Strategy Return"]).cumprod() * 100

# --- Strategie: SPX wird gehalten, wenn Nasdaq > Nasdaq_SMA_200 ---
spx_strategy_returns = []
for i in range(len(df)):
    if i == 0 or pd.isna(df.iloc[i - 1]["Nasdaq_SMA_200"]):
        spx_strategy_returns.append(0)
    else:
        nasdaq_yesterday = df.iloc[i - 1]["Price"]
        nasdaq_sma_yesterday = df.iloc[i - 1]["Nasdaq_SMA_200"]
        daily_ret_spx = df.iloc[i]["SPX_Return"]

        if nasdaq_yesterday > nasdaq_sma_yesterday:
            spx_strategy_returns.append(3 * daily_ret_spx)
        else:
            spx_strategy_returns.append(0)

df["SPX Strategy Return"] = spx_strategy_returns
df["Portfolio Value SPX"] = (1 + df["SPX Strategy Return"]).cumprod() * 100

# --- Ergebnisse anzeigen ---
print("\nSPX Portfolio:\n", df[["Date", "SPX", "Portfolio Value SPX"]].tail(10))
print("\nNasdaq Portfolio:\n", df[["Date", "Price", "Portfolio Value Nasdaq"]].tail(10))


SPX Portfolio:
             Date      SPX  Portfolio Value SPX
9821  03/24/2025  5718.08         61644.628068
9822  03/25/2025  5760.42         61644.628068
9823  03/26/2025  5694.41         61644.628068
9824  03/27/2025  5670.94         61644.628068
9825  03/28/2025  5572.42         61644.628068
9826  03/31/2025  5488.73         61644.628068
9827  04/01/2025  5558.52         61644.628068
9828  04/02/2025  5571.48         61644.628068
9829  04/03/2025  5390.83         61644.628068
9830  04/04/2025  5069.90         61644.628068

Nasdaq Portfolio:
             Date     Price  Portfolio Value Nasdaq
9821  03/24/2025  201.8045           748860.687430
9822  03/25/2025  202.8783           748860.687430
9823  03/26/2025  199.1699           707795.551731
9824  03/27/2025  197.9862           707795.551731
9825  03/28/2025  192.8140           707795.551731
9826  03/31/2025  192.7845           707795.551731
9827  04/01/2025  194.3642           707795.551731
9828  04/02/2025  195.8178           7

In [100]:
merged_df = df[[
    "Date",
    "SPX",
    "SPX_SMA_200",
    "Price",
    "Nasdaq_Return",
    "Portfolio Value Nasdaq"
]]
merged_df

,Date,SPX,SPX_SMA_200,Price,Nasdaq_Return,Portfolio Value Nasdaq
0,1986-04-01,235.139999,NaN,1.4845,NaN,100.000000
1,1986-04-02,235.710007,NaN,1.4892,0.003166,100.000000
2,1986-04-03,232.470001,NaN,1.4871,-0.001410,100.000000
3,1986-04-04,228.690002,NaN,1.4703,-0.011297,100.000000
4,1986-04-07,228.630005,NaN,1.4606,-0.006597,100.000000
...,...,...,...,...,...,...
9826,03/31/2025,5488.730000,5729.26255,192.7845,-0.000153,707795.551731
9827,04/01/2025,5558.520000,5730.00950,194.3642,0.008194,707795.551731
9828,04/02/2025,5571.480000,5730.85435,195.8178,0.007479,707795.551731
9829,04/03/2025,5390.830000,5730.78975,185.2147,-0.054148,707795.551731


### Cross signaling und 0.98

In [103]:
import pandas as pd

# CSV einlesen
df = pd.read_csv("data/SPX&Nasdaq.csv", sep=";", parse_dates=["Date"])

# Sicherstellen, dass SPX und Price numerisch sind
if df["SPX"].dtype == object:
    df["SPX"] = df["SPX"].str.replace(",", "").astype(float)

if df["Price"].dtype == object:
    df["Price"] = df["Price"].str.replace(",", "").astype(float)

df['Price'] =df['Price'] /100
# Berechne SMAs und Returns für beide Indizes
df["SPX_SMA_200"] = df["SPX"].rolling(window=200).mean()
df["SPX_SMA_200"]=df["SPX_SMA_200"]*0.98
df["Nasdaq_SMA_200"] = df["Price"].rolling(window=200).mean()
df["Nasdaq_SMA_200"] =df["Nasdaq_SMA_200"] *0.98

df["SPX_Return"] = df["SPX"].pct_change()
df["Nasdaq_Return"] = df["Price"].pct_change()

# --- Strategie: Nasdaq wird gehalten, wenn SPX > SPX_SMA_200 ---
nasdaq_strategy_returns = []
for i in range(len(df)):
    if i == 0 or pd.isna(df.iloc[i - 1]["SPX_SMA_200"]):
        nasdaq_strategy_returns.append(0)
    else:
        spx_yesterday = df.iloc[i - 1]["SPX"]
        spx_sma_yesterday = df.iloc[i - 1]["SPX_SMA_200"]
        daily_ret_nasdaq = df.iloc[i]["Nasdaq_Return"]

        if spx_yesterday > spx_sma_yesterday:
            nasdaq_strategy_returns.append(3 * daily_ret_nasdaq)
        else:
            nasdaq_strategy_returns.append(0)

df["Nasdaq Strategy Return"] = nasdaq_strategy_returns
df["Portfolio Value Nasdaq"] = (1 + df["Nasdaq Strategy Return"]).cumprod() * 100

# --- Strategie: SPX wird gehalten, wenn Nasdaq > Nasdaq_SMA_200 ---
spx_strategy_returns = []
for i in range(len(df)):
    if i == 0 or pd.isna(df.iloc[i - 1]["Nasdaq_SMA_200"]):
        spx_strategy_returns.append(0)
    else:
        nasdaq_yesterday = df.iloc[i - 1]["Price"]
        nasdaq_sma_yesterday = df.iloc[i - 1]["Nasdaq_SMA_200"]
        daily_ret_spx = df.iloc[i]["SPX_Return"]

        if nasdaq_yesterday > nasdaq_sma_yesterday:
            spx_strategy_returns.append(3 * daily_ret_spx)
        else:
            spx_strategy_returns.append(0)

df["SPX Strategy Return"] = spx_strategy_returns
df["Portfolio Value SPX"] = (1 + df["SPX Strategy Return"]).cumprod() * 100

# --- Ergebnisse anzeigen ---
print("\nSPX Portfolio:\n", df[["Date", "SPX", "Portfolio Value SPX"]].tail(10))
print("\nNasdaq Portfolio:\n", df[["Date", "Price", "Portfolio Value Nasdaq"]].tail(10))


SPX Portfolio:
             Date      SPX  Portfolio Value SPX
9821  03/24/2025  5718.08        121253.954215
9822  03/25/2025  5760.42        123947.459249
9823  03/26/2025  5694.41        119686.430478
9824  03/27/2025  5670.94        118206.536763
9825  03/28/2025  5572.42        118206.536763
9826  03/31/2025  5488.73        118206.536763
9827  04/01/2025  5558.52        118206.536763
9828  04/02/2025  5571.48        118206.536763
9829  04/03/2025  5390.83        118206.536763
9830  04/04/2025  5069.90        118206.536763

Nasdaq Portfolio:
             Date     Price  Portfolio Value Nasdaq
9821  03/24/2025  201.8045            2.715591e+06
9822  03/25/2025  202.8783            2.758940e+06
9823  03/26/2025  199.1699            2.607649e+06
9824  03/27/2025  197.9862            2.561156e+06
9825  03/28/2025  192.8140            2.360432e+06
9826  03/31/2025  192.7845            2.360432e+06
9827  04/01/2025  194.3642            2.360432e+06
9828  04/02/2025  195.8178            

In [96]:
import pandas as pd

# CSV einlesen
df = pd.read_csv("data/SPX&Nasdaq.csv", sep=";", parse_dates=["Date"])

# Sicherstellen, dass SPX und Price numerisch sind
if df["SPX"].dtype == object:
    df["SPX"] = df["SPX"].str.replace(",", "").astype(float)

if df["Price"].dtype == object:
    df["Price"] = df["Price"].str.replace(",", "").astype(float)

# Berechne SMAs und Returns für beide Indizes
df["SPX_SMA_200"] = df["SPX"].rolling(window=200).mean()
df["Nasdaq_SMA_200"] = df["Price"].rolling(window=200).mean()

df["SPX_Return"] = df["SPX"].pct_change()
df["Nasdaq_Return"] = df["Price"].pct_change()

# --- Strategie: Nasdaq wird gehalten, wenn SPX > SPX_SMA_200 ---
nasdaq_strategy_returns = []
for i in range(len(df)):
    if i == 0 or pd.isna(df.iloc[i - 1]["SPX_SMA_200"]):
        nasdaq_strategy_returns.append(0)
    else:
        spx_yesterday = df.iloc[i - 1]["SPX"]
        spx_sma_yesterday = df.iloc[i - 1]["SPX_SMA_200"]
        daily_ret_nasdaq = df.iloc[i]["Nasdaq_Return"]

        if spx_yesterday > spx_sma_yesterday:
            nasdaq_strategy_returns.append(3 * daily_ret_nasdaq)
        else:
            nasdaq_strategy_returns.append(0)

df["Nasdaq Strategy Return"] = nasdaq_strategy_returns
df["Portfolio Value Nasdaq"] = (1 + df["Nasdaq Strategy Return"]).cumprod() * 100

# --- Strategie: SPX wird gehalten, wenn Nasdaq > Nasdaq_SMA_200 ---
spx_strategy_returns = []
for i in range(len(df)):
    if i == 0 or pd.isna(df.iloc[i - 1]["Nasdaq_SMA_200"]):
        spx_strategy_returns.append(0)
    else:
        nasdaq_yesterday = df.iloc[i - 1]["Price"]
        nasdaq_sma_yesterday = df.iloc[i - 1]["Nasdaq_SMA_200"]
        daily_ret_spx = df.iloc[i]["SPX_Return"]

        if nasdaq_yesterday > nasdaq_sma_yesterday:
            spx_strategy_returns.append(3 * daily_ret_spx)
        else:
            spx_strategy_returns.append(0)

df["SPX Strategy Return"] = spx_strategy_returns
df["Portfolio Value SPX"] = (1 + df["SPX Strategy Return"]).cumprod() * 100

# --- Ergebnisse anzeigen ---
print("\nSPX Portfolio:\n", df[["Date", "SPX", "Portfolio Value SPX"]].tail(10))
print("\nNasdaq Portfolio:\n", df[["Date", "Price", "Portfolio Value Nasdaq"]].tail(10))



SPX Portfolio:
             Date      SPX  Portfolio Value SPX
9821  03/24/2025  5718.08         23944.319533
9822  03/25/2025  5760.42         23944.319533
9823  03/26/2025  5694.41         23944.319533
9824  03/27/2025  5670.94         23944.319533
9825  03/28/2025  5572.42         23944.319533
9826  03/31/2025  5488.73         23944.319533
9827  04/01/2025  5558.52         23944.319533
9828  04/02/2025  5571.48         23944.319533
9829  04/03/2025  5390.83         23944.319533
9830  04/04/2025  5069.90         23944.319533

Nasdaq Portfolio:
             Date     Price  Portfolio Value Nasdaq
9821  03/24/2025  19753.97            1.014440e+12
9822  03/25/2025  20180.45            1.014440e+12
9823  03/26/2025  20287.83            1.030634e+12
9824  03/27/2025  19916.99            1.030634e+12
9825  03/28/2025  19798.62            1.030634e+12
9826  03/31/2025  19281.40            1.030634e+12
9827  04/01/2025  19278.45            1.030634e+12
9828  04/02/2025  19436.42            

In [ ]:
import pandas as pd

# Kopie des ursprünglichen DataFrames
df = kgv_on_spx_days.copy()

# 1. 200-Tage-Linie berechnen
df["SMA_200"] = df["Price"].rolling(window=200).mean()

# 2. Tägliche Rendite berechnen
df["Daily Return"] = df["Price"].pct_change()

# 3. Portfoliowert berechnen mit angepasster Reallokationslogik
initial_value = 100
portfolio = [initial_value]

# Tracking der Investitionslage (True = investiert)
invested = False

for i in range(1, len(df)):
    if i < 200:
        portfolio.append(initial_value)
        continue

    price_yesterday = df.iloc[i - 1]["Price"]
    sma_yesterday = df.iloc[i - 1]["SMA_200"]
    price_today = df.iloc[i]["Price"]
    sma_today = df.iloc[i]["SMA_200"]
    ret_today = df.iloc[i]["Daily Return"]

    # Default: kein Reallokationstag
    adjusted_return = ret_today

    # Prüfe, ob heute ein Abwärts-Durchbruch ist (Wechsel von investiert auf nicht investiert)
    if invested and (price_yesterday > sma_yesterday and price_today < sma_today):
        # Reallokation nach unten -> verwende statt ret_today die Differenz zum SMA
        adjusted_return = (sma_today - price_yesterday) / price_yesterday
        invested = False  # Ab jetzt nicht mehr investiert
    elif price_yesterday > sma_yesterday:
        invested = True
    else:
        invested = False

    # Portfolio-Update basierend auf Investitionsstatus
    if invested:
        new_value = portfolio[-1] * (1 + 3 * adjusted_return)
    else:
        new_value = portfolio[-1] * (1 + 0.0001)

    portfolio.append(new_value)

# 4. Neue Spalte anhängen
df["Portfolio Value"] = portfolio

# 5. Rückgabe
df.tail(10)



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

spx = pd.read_csv(
    "data/SPX.csv",
    sep=";",
    usecols=["Date", "Close"],
    parse_dates=["Date"]
)
spx["Date"] = pd.to_datetime(spx["Date"], format="mixed", dayfirst=False, errors="coerce")

spx = spx[["Date", "Close"]].rename(columns={"Close": "Price"})
spx = spx.sort_values("Date")
spx = spx.iloc[10000:].reset_index(drop=True)
spx["Price"] = spx["Price"].str.replace(",", "").astype(float)
spx["Change"] = spx["Price"].pct_change()


# Startwert für das Portfolio
initial_value = 100

# Hebel-Werte
leverage_factors = [1, 2, 3, 4]

# Für jeden Hebel eine neue Spalte berechnen
for lev in leverage_factors:
    spx[f"Leverage_{lev}x"] = initial_value * (1 + lev * spx["Change"]).cumprod()
spx.head(10)

In [ ]:
plt.figure(figsize=(12, 6))
for lev in leverage_factors:
    plt.plot(spx["Date"], spx[f"Leverage_{lev}x"], label=f"{lev}x Hebel")

plt.title("Hebel-Strategien auf SPX (Start bei 100)")
plt.xlabel("Datum")
plt.ylabel("Portfoliowert")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 1. CSV laden
# Achte darauf, dass der Pfad stimmt
data = pd.read_csv(
    'data/BTC.csv',       # Pfad zu deiner CSV
    sep=',',              # Kommagetrennt
    quotechar='"',        # Anführungszeichen
    thousands=','         # Entfernt das Komma in 77,355.0
)

# Schau dir an, welche Spalten jetzt drin sind
print("Spalten:", data.columns)

# 2. Datumsformat parsen
# Deine Beispiel-Daten sind im Format MM/DD/YYYY, z.B. 04/07/2025
data['Date'] = pd.to_datetime(data['Date'], format='%m/%d/%Y')

# Sortiere die Daten chronologisch
data = data.sort_values('Date').reset_index(drop=True)

# 3. Technische Indikatoren
# Wir nutzen "Price" als Kurs
data['SMA200'] = data['Price'].rolling(window=200).mean()
data['EMA10']  = data['Price'].ewm(span=10, adjust=False).mean()
data['EMA50']  = data['Price'].ewm(span=50, adjust=False).mean()
data['EMA200'] = data['Price'].ewm(span=200, adjust=False).mean()

# Tägliche BTC-Rendite (prozentuale Änderung)
data['BTC_return'] = data['Price'].pct_change()

# 4. Portfolio-Simulation

# Bedingung für Portfolio 2:
# BTC-Preis am Vortag > SMA200 am Vortag
data['cond_port2'] = (data['Price'].shift(1) > data['SMA200'].shift(1)).astype(int)

# Bedingung für Portfolio 3:
# EMA50 am Vortag > EMA200 am Vortag
data['cond_port3'] = (data['EMA50'].shift(1) > data['EMA200'].shift(1)).astype(int)

# Berechne tägliche Renditen der drei Portfolios
data['port1_return'] = data['BTC_return']                          # Portfolio 1: normaler BTC-Return
data['port2_return'] = data['BTC_return'] * 2 * data['cond_port2'] # Portfolio 2: 2× Return, wenn cond_port2=1, sonst 0
data['port3_return'] = data['BTC_return'] * 2 * data['cond_port3'] # Portfolio 3: 2× Return, wenn cond_port3=1, sonst 0

# Ersetze NaN-Werte in den Renditen (z. B. in der ersten Zeile) mit 0
data[['port1_return', 'port2_return', 'port3_return']] = data[['port1_return', 'port2_return', 'port3_return']].fillna(0)

# Kumulativer Wert (Start: 100)
data['port1_value'] = 100 * (1 + data['port1_return']).cumprod()
data['port2_value'] = 100 * (1 + data['port2_return']).cumprod()
data['port3_value'] = 100 * (1 + data['port3_return']).cumprod()

# 5. Ergebnisse ansehen
print(data[['Date', 'Price', 'SMA200', 'EMA10', 'EMA50', 'EMA200',
            'BTC_return', 'port1_value', 'port2_value', 'port3_value']].head(10))

# Plot
plt.figure(figsize=(12, 6))
plt.plot(data['Date'], data['port1_value'], label='Portfolio 1 (BTC Return)')
plt.plot(data['Date'], data['port2_value'], label='Portfolio 2 (2× Return, Price > SMA200)')
plt.plot(data['Date'], data['port3_value'], label='Portfolio 3 (2× Return, EMA50 > EMA200)')
plt.xlabel('Datum')
plt.ylabel('Portfoliowert')
plt.title('Portfolio-Simulation basierend auf BTC-Daten')
plt.legend()
plt.tight_layout()
plt.show()




In [ ]:
data.tail()